# WINGS Production Campaign Testing

This notebook tests the production campaign infrastructure for large-scale optimization.

**Tests covered:**
- CampaignConfig creation and serialization
- OptimizationManager initialization
- Single run execution
- Checkpointing and resume
- Full campaign execution
- Results analysis and statistics
- Strategy distribution verification

In [1]:
# ============================================================================
# FRESH START - Run this FIRST after kernel restart
# ============================================================================
import sys
from pathlib import Path

# Clean up sys.path - remove any gaussian_state_optimizer paths
sys.path = [p for p in sys.path if 'gaussian_state_optimizer' not in p and 'optimization' not in p]

# Add the correct path
src_path = Path.cwd() / 'optimization-files' / 'src'
sys.path.insert(0, str(src_path))

print("sys.path (first 3):")
for p in sys.path[:3]:
    print(f"  {p}")

# Now verify the package exists at that location
init_file = src_path / 'gaussian_state_optimizer' / '__init__.py'
print(f"\n__init__.py exists: {init_file.exists()}")

if init_file.exists():
    print(f"✓ Package found at: {src_path / 'gaussian_state_optimizer'}")
else:
    print(f"✗ Package NOT found. Check your directory structure.")
    print(f"  Looking for: {init_file}")

sys.path (first 3):
  /home/jmc03846/WINGS_debug/optimization-files/src
  /home/jmc03846/WINGS_debug
  

__init__.py exists: True
✓ Package found at: /home/jmc03846/WINGS_debug/optimization-files/src/gaussian_state_optimizer


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tempfile
import shutil
from pathlib import Path

from gaussian_state_optimizer import (
    get_backend_info,
    print_backend_info,
)
from gaussian_state_optimizer.config import CampaignConfig
from gaussian_state_optimizer.campaign import (
    OptimizationManager,
    run_production_campaign,
    list_campaigns,
)
from gaussian_state_optimizer.results import CampaignResults, RunResult

print_backend_info()

backend_info = get_backend_info()
USE_CUSTATEVEC = backend_info['custatevec']

Detected HPC scratch directory: /scratch/jmc03846
Created/verified directory: /home/jmc03846/lib
Created/verified directory: /scratch/jmc03846/coefficient_cache
Created/verified directory: /scratch/jmc03846/simulation_output
Created/verified directory: /scratch/jmc03846/simulation_data
Created/verified directory: /scratch/jmc03846/optimization_checkpoints
Created/verified directory: /scratch/jmc03846/optimization_campaigns
Gaussian State Optimizer - Backend Information
  CPU (Qiskit Statevector): ✓ Available
  GPU (Qiskit Aer):         ✓ Available
  cuStateVec:               ✓ Available


In [3]:
# Create temporary directories for testing
TEST_DIR = tempfile.mkdtemp(prefix='wings_campaign_test_')
OUTPUT_DIR = os.path.join(TEST_DIR, 'outputs')
CHECKPOINT_DIR = os.path.join(TEST_DIR, 'checkpoints')

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"Test directory: {TEST_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Checkpoint directory: {CHECKPOINT_DIR}")

Test directory: /tmp/wings_campaign_test_iy52rv_h
Output directory: /tmp/wings_campaign_test_iy52rv_h/outputs
Checkpoint directory: /tmp/wings_campaign_test_iy52rv_h/checkpoints


## 1. CampaignConfig Creation and Serialization

In [4]:
# Create campaign configuration
config = CampaignConfig(
    n_qubits=6,
    sigma=0.5,
    total_runs=20,  # Small number for testing
    runs_per_batch=5,
    target_infidelity=1e-6,
    acceptable_infidelity=1e-4,
    
    # Strategy distribution
    strategy_weights={
        'smart': 0.3,
        'gaussian_product': 0.2,
        'random': 0.4,
        'perturb_best': 0.1,
    },
    
    # Per-run settings
    max_iter_per_run=200,
    tolerance_per_run=1e-10,
    use_ultra_precision=False,  # Faster for testing
    ultra_precision_time_limit=30,
    
    # GPU settings
    use_gpu=False,
    use_custatevec=USE_CUSTATEVEC,
    
    # Checkpointing
    checkpoint_interval=5,
    checkpoint_dir=CHECKPOINT_DIR,
    
    # Output
    campaign_name='test_campaign',
    output_dir=OUTPUT_DIR,
    verbose=1,
    
    # Seed for reproducibility
    base_seed=42,
)

print("CampaignConfig created:")
print(f"  n_qubits: {config.n_qubits}")
print(f"  sigma: {config.sigma}")
print(f"  total_runs: {config.total_runs}")
print(f"  target_infidelity: {config.target_infidelity}")
print(f"  target_fidelity: {config.target_fidelity}")
print(f"  campaign_name: {config.campaign_name}")

CampaignConfig created:
  n_qubits: 6
  sigma: 0.5
  total_runs: 20
  target_infidelity: 1e-06
  target_fidelity: 0.999999
  campaign_name: test_campaign


In [5]:
# Test serialization
config_path = os.path.join(OUTPUT_DIR, 'config_test.json')
saved_path = config.save(config_path)

print(f"Config saved to: {saved_path}")
assert os.path.exists(saved_path), "Config file not created"

# Load and verify
loaded_config = CampaignConfig.load(saved_path)

assert loaded_config.n_qubits == config.n_qubits, "n_qubits mismatch"
assert loaded_config.sigma == config.sigma, "sigma mismatch"
assert loaded_config.total_runs == config.total_runs, "total_runs mismatch"

print("\n✓ Config serialization/deserialization verified")

Config saved to: /tmp/wings_campaign_test_iy52rv_h/outputs/config_test.json

✓ Config serialization/deserialization verified


In [6]:
# Test strategy assignment determinism
print("\nStrategy assignment test (should be deterministic):")
strategies_1 = [config.get_strategy_for_run(i) for i in range(10)]
strategies_2 = [config.get_strategy_for_run(i) for i in range(10)]

assert strategies_1 == strategies_2, "Strategy assignment not deterministic"

print(f"  Run 0-9 strategies: {strategies_1}")
print("\n✓ Strategy assignment is deterministic")


Strategy assignment test (should be deterministic):
  Run 0-9 strategies: [np.str_('gaussian_product'), np.str_('smart'), np.str_('random'), np.str_('perturb_best'), np.str_('random'), np.str_('smart'), np.str_('smart'), np.str_('gaussian_product'), np.str_('gaussian_product'), np.str_('random')]

✓ Strategy assignment is deterministic


## 2. OptimizationManager Initialization

In [7]:
# Create manager
manager = OptimizationManager(config)

print("OptimizationManager created:")
print(f"  Config: {manager.config.campaign_name}")
print(f"  Results initialized: {manager.results is not None}")
print(f"  Completed runs: {len(manager._completed_runs)}")

No checkpoint found, starting fresh
OptimizationManager initialized for campaign: test_campaign
  Total runs: 20
  Target infidelity: 1e-06
  Output directory: /tmp/wings_campaign_test_iy52rv_h/outputs
OptimizationManager created:
  Config: test_campaign
  Results initialized: True
  Completed runs: 0


In [8]:
# Test optimizer config creation
opt_config = manager._create_optimizer_config()

print("\nOptimizer config from campaign:")
print(f"  n_qubits: {opt_config.n_qubits}")
print(f"  sigma: {opt_config.sigma}")
print(f"  verbose: {opt_config.verbose}")
print(f"  use_custatevec: {opt_config.use_custatevec}")

assert opt_config.n_qubits == config.n_qubits, "n_qubits mismatch"
assert opt_config.sigma == config.sigma, "sigma mismatch"
assert opt_config.verbose == False, "verbose should be False for batch runs"

print("\n✓ Optimizer config creation verified")


Optimizer config from campaign:
  n_qubits: 6
  sigma: 0.5
  verbose: False
  use_custatevec: True

✓ Optimizer config creation verified


## 3. Single Run Execution

In [9]:
# Test single run
print("Testing single optimization run...")

start = time.time()
result = manager._run_single_optimization(run_id=0)
elapsed = time.time() - start

print(f"\nSingle run completed in {elapsed:.1f}s:")
print(f"  run_id: {result.run_id}")
print(f"  strategy: {result.strategy}")
print(f"  seed: {result.seed}")
print(f"  fidelity: {result.fidelity:.10f}")
print(f"  infidelity: {result.infidelity:.6e}")
print(f"  success: {result.success}")
print(f"  n_evaluations: {result.n_evaluations}")
print(f"  time_seconds: {result.time_seconds:.1f}")

assert result.run_id == 0, "run_id should be 0"
assert result.success == True, "Run should succeed"
assert 0 <= result.fidelity <= 1, "Fidelity out of range"
assert result.params is not None, "Params should be saved"
assert result.params.shape == (config.n_qubits**2,), "Wrong params shape"

print("\n✓ Single run execution verified")

Testing single optimization run...

Initializing cuStateVec acceleration...
  ✓ cuStateVec initialized
    Precision: double
    Batch simulators: 4

Adam Optimization (lr=0.01, max_steps=500)
--------------------------------------------------
  Step     0: F=0.1060504812, |∇|=3.38e-01, lr=0.0100
  Step   100: F=0.9733323533, |∇|=8.36e-02, lr=0.0051
  Step   200: F=0.9825840676, |∇|=4.90e-02, lr=0.0100
  Step   300: F=0.9939522764, |∇|=1.16e-02, lr=0.0075
  Time limit reached at step 361

Adam complete: F=0.995647591639 in 600.4s (362 steps)

Single run completed in 600.9s:
  run_id: 0
  strategy: gaussian_product
  seed: 42
  fidelity: 0.9956475916
  infidelity: 4.352408e-03
  success: True
  n_evaluations: 0
  time_seconds: 600.8

✓ Single run execution verified


## 4. Results Management

In [10]:
# Test adding results
manager.results.add_result(result)
manager._completed_runs.add(0)

print("After adding first result:")
print(f"  Completed runs: {len(manager._completed_runs)}")
print(f"  Best result: {manager.results.best_result.fidelity:.10f}")

# Run a few more
print("\nRunning additional optimizations...")
for run_id in range(1, 5):
    result = manager._run_single_optimization(run_id)
    manager.results.add_result(result)
    manager._completed_runs.add(run_id)
    print(f"  Run {run_id}: F={result.fidelity:.8f} ({result.strategy})")

print(f"\nBest fidelity so far: {manager.results.best_result.fidelity:.10f}")

  ★ New best: Run 0, F=0.995647591639, 1-F=4.352e-03
After adding first result:
  Completed runs: 1
  Best result: 0.9956475916

Running additional optimizations...

Initializing cuStateVec acceleration...
  ✓ cuStateVec initialized
    Precision: double
    Batch simulators: 4

Adam Optimization (lr=0.01, max_steps=500)
--------------------------------------------------
  Step     0: F=0.0187346896, |∇|=1.12e-01, lr=0.0100
  Step   100: F=0.9685943534, |∇|=1.20e-01, lr=0.0051
  Step   200: F=0.9872290525, |∇|=4.50e-02, lr=0.0100
  Step   300: F=0.9944961920, |∇|=9.60e-03, lr=0.0075
  Step   400: F=0.9960648775, |∇|=7.37e-03, lr=0.0027

Adam complete: F=0.996300494205 in 423.7s (500 steps)
  ★ New best: Run 1, F=0.996300494205, 1-F=3.700e-03
  Run 1: F=0.99630049 (smart)

Initializing cuStateVec acceleration...
  ✓ cuStateVec initialized
    Precision: double
    Batch simulators: 4

Adam Optimization (lr=0.01, max_steps=500)
--------------------------------------------------
  Step   

In [11]:
# Test statistics
stats = manager.results.get_statistics()

print("Statistics after 5 runs:")
print(f"  total_runs: {stats['total_runs']}")
print(f"  successful_runs: {stats['successful_runs']}")
print(f"  success_rate: {stats['success_rate']*100:.0f}%")
print(f"  best_fidelity: {stats['best_fidelity']:.10f}")
print(f"  mean_fidelity: {stats['mean_fidelity']:.8f}")
print(f"  std_fidelity: {stats['std_fidelity']:.6e}")

assert stats['total_runs'] == 5, "Should have 5 runs"
assert stats['successful_runs'] == 5, "All runs should succeed"

print("\n✓ Statistics computation verified")

Statistics after 5 runs:
  total_runs: 5
  successful_runs: 5
  success_rate: 100%
  best_fidelity: 0.9996532281
  mean_fidelity: 0.99222369
  std_fidelity: 9.492245e-03

✓ Statistics computation verified


## 5. Checkpointing and Resume

In [12]:
# Save checkpoint
print("Saving checkpoint...")
manager._save_checkpoint()

checkpoint_file = os.path.join(CHECKPOINT_DIR, 'test_campaign', 'checkpoint.pkl')
print(f"Checkpoint file exists: {os.path.exists(checkpoint_file)}")

Saving checkpoint...
Checkpoint file exists: False


In [13]:
# Create new manager with resume
print("\nCreating new manager with resume=True...")

config_resume = CampaignConfig(
    n_qubits=6,
    sigma=0.5,
    total_runs=20,
    runs_per_batch=5,
    target_infidelity=1e-6,
    max_iter_per_run=200,
    use_ultra_precision=False,
    use_custatevec=USE_CUSTATEVEC,
    checkpoint_dir=CHECKPOINT_DIR,
    campaign_name='test_campaign',
    output_dir=OUTPUT_DIR,
    verbose=1,
    base_seed=42,
    resume_from_checkpoint=True,  # KEY: Resume!
)

manager_resumed = OptimizationManager(config_resume)

print(f"Resumed with {len(manager_resumed._completed_runs)} completed runs")
print(f"Best fidelity from checkpoint: {manager_resumed.results.best_result.fidelity:.10f}")

assert len(manager_resumed._completed_runs) == 5, "Should have 5 completed runs"

print("\n✓ Checkpoint resume verified")


Creating new manager with resume=True...
Resumed from checkpoint: 5 runs completed
  Best fidelity so far: 0.999653228088
OptimizationManager initialized for campaign: test_campaign
  Total runs: 20
  Target infidelity: 1e-06
  Output directory: /tmp/wings_campaign_test_iy52rv_h/outputs
Resumed with 5 completed runs
Best fidelity from checkpoint: 0.9996532281

✓ Checkpoint resume verified


## 6. Full Campaign Execution

In [14]:
# Clean start for full campaign test
CAMPAIGN_OUTPUT = os.path.join(TEST_DIR, 'full_campaign_output')
CAMPAIGN_CHECKPOINT = os.path.join(TEST_DIR, 'full_campaign_checkpoint')

os.makedirs(CAMPAIGN_OUTPUT, exist_ok=True)
os.makedirs(CAMPAIGN_CHECKPOINT, exist_ok=True)

full_config = CampaignConfig(
    n_qubits=6,
    sigma=0.5,
    total_runs=15,  # Small campaign
    runs_per_batch=5,
    target_infidelity=1e-6,
    max_iter_per_run=300,
    use_ultra_precision=False,
    use_custatevec=USE_CUSTATEVEC,
    checkpoint_interval=5,
    checkpoint_dir=CAMPAIGN_CHECKPOINT,
    campaign_name='full_test_campaign',
    output_dir=CAMPAIGN_OUTPUT,
    verbose=1,
    base_seed=123,
)

In [ ]:
# Run full campaign
print("Running full campaign...")
print("="*60)

full_manager = OptimizationManager(full_config)

start_time = time.time()
results = full_manager.run_campaign()
total_time = time.time() - start_time

print(f"\nCampaign completed in {total_time:.1f}s")

Running full campaign...
No checkpoint found, starting fresh
OptimizationManager initialized for campaign: full_test_campaign
  Total runs: 15
  Target infidelity: 1e-06
  Output directory: /tmp/wings_campaign_test_iy52rv_h/full_campaign_output
STARTING OPTIMIZATION CAMPAIGN: full_test_campaign
Runs to complete: 15/15

Batch 1/3 (runs 0-4)
  Starting run 0/15... 
Initializing GPU acceleration...
  ✓ GPU backend initialized
    Precision: double
    Available devices: ['GPU']
  GPU acceleration enabled

Initializing cuStateVec acceleration...
  ✓ cuStateVec initialized
    Precision: double
    Batch simulators: 4

Adam Optimization (lr=0.01, max_steps=500)
--------------------------------------------------
  Step     0: F=0.0071918111, |∇|=6.99e-02, lr=0.0100
  Step   100: F=0.8536548499, |∇|=2.16e-01, lr=0.0051
  Step   200: F=0.9502489051, |∇|=7.80e-02, lr=0.0100
  Step   300: F=0.9833614885, |∇|=2.71e-02, lr=0.0075
  Step   400: F=0.9957895796, |∇|=4.71e-03, lr=0.0027

Adam complete

In [ ]:
# Print summary
results.print_summary()

## 7. Results Analysis

In [ ]:
# Detailed analysis
stats = results.get_statistics()

print("Detailed Campaign Statistics")
print("="*60)

print(f"\nRun Statistics:")
print(f"  Total runs: {stats['total_runs']}")
print(f"  Successful: {stats['successful_runs']}")
print(f"  Failed: {stats['failed_runs']}")
print(f"  Success rate: {stats['success_rate']*100:.1f}%")

print(f"\nFidelity Statistics:")
print(f"  Best: {stats['best_fidelity']:.12f}")
print(f"  Mean: {stats['mean_fidelity']:.10f}")
print(f"  Std: {stats['std_fidelity']:.6e}")
print(f"  Median: {stats['median_fidelity']:.10f}")

print(f"\nPercentiles:")
for pct, val in stats['fidelity_percentiles'].items():
    print(f"  {pct}%: {val:.10f}")

In [ ]:
# Strategy breakdown
print("\nStrategy Performance:")
print("-"*60)

for strategy, strat_stats in stats['strategy_breakdown'].items():
    print(f"\n{strategy}:")
    print(f"  Count: {strat_stats['count']}")
    print(f"  Best fidelity: {strat_stats['best_fidelity']:.10f}")
    print(f"  Mean fidelity: {strat_stats['mean_fidelity']:.8f}")
    print(f"  Std: {strat_stats['std_fidelity']:.6e}")

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Fidelity distribution
ax = axes[0, 0]
fidelities = [r.fidelity for r in results.results if r.success]
ax.hist(fidelities, bins=20, edgecolor='black', alpha=0.7)
ax.axvline(x=stats['mean_fidelity'], color='r', linestyle='--', label=f"Mean: {stats['mean_fidelity']:.6f}")
ax.axvline(x=stats['best_fidelity'], color='g', linestyle='-', label=f"Best: {stats['best_fidelity']:.6f}")
ax.set_xlabel('Fidelity')
ax.set_ylabel('Count')
ax.set_title('Fidelity Distribution')
ax.legend()

# 2. Fidelity by strategy
ax = axes[0, 1]
strategy_data = {}
for r in results.results:
    if r.success:
        if r.strategy not in strategy_data:
            strategy_data[r.strategy] = []
        strategy_data[r.strategy].append(r.fidelity)

positions = range(len(strategy_data))
ax.boxplot(strategy_data.values(), positions=positions)
ax.set_xticks(positions)
ax.set_xticklabels(strategy_data.keys(), rotation=45)
ax.set_ylabel('Fidelity')
ax.set_title('Fidelity by Strategy')

# 3. Time per run
ax = axes[1, 0]
times = [r.time_seconds for r in results.results if r.success]
ax.hist(times, bins=15, edgecolor='black', alpha=0.7, color='coral')
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Count')
ax.set_title(f'Optimization Time Distribution (mean: {np.mean(times):.1f}s)')

# 4. Fidelity vs evaluations
ax = axes[1, 1]
evals = [r.n_evaluations for r in results.results if r.success]
fids = [r.fidelity for r in results.results if r.success]
colors = [hash(r.strategy) % 10 for r in results.results if r.success]
scatter = ax.scatter(evals, fids, c=colors, alpha=0.7, cmap='tab10')
ax.set_xlabel('Function Evaluations')
ax.set_ylabel('Fidelity')
ax.set_title('Fidelity vs Evaluations')

plt.tight_layout()
plt.savefig(os.path.join(CAMPAIGN_OUTPUT, 'campaign_analysis.png'), dpi=150)
plt.show()

## 8. Results Persistence

In [ ]:
# Save results
results_path = results.save()
print(f"Results saved to: {results_path}")

# Check associated files
summary_path = results_path.replace('.pkl', '_summary.json')
params_path = results_path.replace('.pkl', '_best_params.npy')

print(f"\nFiles created:")
print(f"  Main results: {os.path.exists(results_path)}")
print(f"  JSON summary: {os.path.exists(summary_path)}")
print(f"  Best params: {os.path.exists(params_path)}")

In [ ]:
# Load results
loaded_results = CampaignResults.load(results_path)

print(f"\nLoaded results:")
print(f"  Best fidelity: {loaded_results.best_result.fidelity:.10f}")
print(f"  Total runs: {len(loaded_results.results)}")

assert loaded_results.best_result.fidelity == results.best_result.fidelity, "Fidelity mismatch"

print("\n✓ Results persistence verified")

In [ ]:
# Load best parameters
best_params = np.load(params_path)

print(f"Best parameters:")
print(f"  Shape: {best_params.shape}")
print(f"  Range: [{best_params.min():.4f}, {best_params.max():.4f}]")

# Verify they reproduce the best fidelity
from gaussian_state_optimizer import GaussianOptimizer, OptimizerConfig

verify_config = OptimizerConfig(
    n_qubits=full_config.n_qubits,
    sigma=full_config.sigma,
    verbose=False,
    use_custatevec=USE_CUSTATEVEC,
)
verify_opt = GaussianOptimizer(verify_config)
verify_fidelity = verify_opt.compute_fidelity(params=best_params)

print(f"\nVerification:")
print(f"  Reported best: {results.best_result.fidelity:.12f}")
print(f"  Recomputed:    {verify_fidelity:.12f}")

assert np.isclose(verify_fidelity, results.best_result.fidelity, rtol=1e-6), "Fidelity mismatch"
print("\n✓ Best parameters reproduce reported fidelity")

## 9. Cleanup

In [ ]:
# Cleanup temporary directories
cleanup = input("Clean up test directory? (y/n): ").strip().lower()

if cleanup == 'y':
    shutil.rmtree(TEST_DIR)
    print(f"Cleaned up: {TEST_DIR}")
else:
    print(f"Test files kept at: {TEST_DIR}")

## Summary

In [ ]:
print("\n" + "="*70)
print("CAMPAIGN TESTING SUMMARY")
print("="*70)

print("\n✓ CampaignConfig creation and serialization")
print("✓ Strategy assignment determinism")
print("✓ OptimizationManager initialization")
print("✓ Single run execution")
print("✓ Results management and statistics")
print("✓ Checkpointing")
print("✓ Resume from checkpoint")
print("✓ Full campaign execution")
print("✓ Results persistence and loading")
print("✓ Best parameters verification")

print("\n" + "="*70)
print(f"Campaign infrastructure is working correctly!")
print(f"Best fidelity achieved: {results.best_result.fidelity:.12f}")
print("="*70)